In [1]:
"""import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
print ("PyTorch Version:", torch.__version__)"""

'import numpy as np\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nimport torch.optim as optim\nfrom torchvision import datasets, transforms\nprint ("PyTorch Version:", torch.__version__)'

In [2]:
"""mnist_data = datasets.MNIST("./mnist_data", train=True, download=True,
                           transform=transforms.Compose([transforms.ToTensor()]))
mnist_data"""

'mnist_data = datasets.MNIST("./mnist_data", train=True, download=True,\n                           transform=transforms.Compose([transforms.ToTensor()]))\nmnist_data'

In [3]:
"""mnist_data[0][0].shape"""

'mnist_data[0][0].shape'

In [4]:
"""data = [d[0].data.cpu().numpy() for d in mnist_data]
print(np.mean(data), np.std(data))"""

'data = [d[0].data.cpu().numpy() for d in mnist_data]\nprint(np.mean(data), np.std(data))'

In [5]:
"""device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data",
                   train=True,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor()]),
                   transforms.Normalize((0.13066062,), (0.30810776,))
                  ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
    pin_memory=True
    )
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./mnist_data",
                   train=False,
                   download=True,
                   transform=transforms.Compose([transforms.ToTensor()]),
                   transforms.Normalize((0.13066062,), (0.30810776,))
                  ),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=1,
    pin_memory=True
    )

lr = 0.01
momentum = 0.5
model = Net().to(device)
optimizer = torch.optim.SGD(model.parameters, lr=lr, momentum=momentum)

num_epochs = 2"""

'device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")\nBATCH_SIZE = 32\ntrain_loader = torch.utils.data.DataLoader(\n    datasets.MNIST("./mnist_data",\n                   train=True,\n                   download=True,\n                   transform=transforms.Compose([transforms.ToTensor()]),\n                   transforms.Normalize((0.13066062,), (0.30810776,))\n                  ),\n    batch_size=BATCH_SIZE,\n    shuffle=True,\n    num_workers=1,\n    pin_memory=True\n    )\ntest_loader = torch.utils.data.DataLoader(\n    datasets.MNIST("./mnist_data",\n                   train=False,\n                   download=True,\n                   transform=transforms.Compose([transforms.ToTensor()]),\n                   transforms.Normalize((0.13066062,), (0.30810776,))\n                  ),\n    batch_size=BATCH_SIZE,\n    shuffle=True,\n    num_workers=1,\n    pin_memory=True\n    )\n\nlr = 0.01\nmomentum = 0.5\nmodel = Net().to(device)\noptimizer = torch.optim.SGD(mode

In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt
import time
import os
import copy
import cv2
print ("PyTorch Version:", torch.__version__)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PyTorch Version: 1.9.0+cu111


In [7]:
data_dir = "./hymenoptera_data"
model_name = "resnet"
num_classes = 2
batch_size = 32
num_epochs = 15
feature_extract = True

input_size = 224

In [8]:
"""all_imgs = datasets.ImageFolder(os.path.join(data_dir, "train"),transforms.Compose([
                    transforms.RandomResizedCrop(input_size),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.ToTensor()])
                    )
train_loader = torch.utils.data.DataLoader(all_imgs, batch_size=batch_size, shuffle=True, num_workers=2)"""

'all_imgs = datasets.ImageFolder(os.path.join(data_dir, "train"),transforms.Compose([\n                    transforms.RandomResizedCrop(input_size),\n                    transforms.RandomHorizontalFlip(p=0.5),\n                    transforms.ToTensor()])\n                    )\ntrain_loader = torch.utils.data.DataLoader(all_imgs, batch_size=batch_size, shuffle=True, num_workers=2)'

In [9]:
data_transforms = {
    "train": transforms.Compose([
                    transforms.RandomResizedCrop(input_size),
                    transforms.RandomHorizontalFlip(p=0.5),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    "val": transforms.Compose([
                    transforms.Resize(input_size),
                    transforms.CenterCrop(input_size),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                         data_transforms[x]) for x in ["train", "val"]}

dataloader_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                              batch_size=batch_size,
                                              shuffle=True,
                                              num_workers=4) for x in ["train", "val"]}

/opt/conda/lib/python3.8/site-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [10]:
"""img = next(iter(train_loader))[0]
img.shape"""

'img = next(iter(train_loader))[0]\nimg.shape'

In [11]:
"""unloader = transforms.ToPILImage()
plt.ion

x = img[8]
plt.imshow(unloader(x.squeeze(0)))"""

'unloader = transforms.ToPILImage()\nplt.ion\n\nx = img[8]\nplt.imshow(unloader(x.squeeze(0)))'

In [12]:
def set_parameter_requires_grad(model, feature_extract):
    if feature_extract:
        for para in model.parameters():
            para.requires_grad = False

def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    if model_name == "resnet":
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # 更改fc层：1000->2
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
    else:
        print("model not implemented")
        return None, None
    return model_ft, input_size

model_ft, input_size = initialize_model(model_name,
                                       num_classes,
                                       feature_extract,
                                       use_pretrained=True)

In [13]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
model_ft.layer1[0].conv1.weight.requires_grad

False

In [15]:
model_ft.fc.weight.requires_grad

True

In [16]:
def train(model, dataloaders, loss_fn, optimizer, num_epochs):
    best_acc = 0.
    best_model_wts = copy.deepcopy(model.state_dict())
    epoch_loss = 0.
    epoch_correct = 0.
    val_acc_history = []
    for epoch in range(num_epochs):
        for phase in ["train", "val"]:
            running_loss = 0.
            running_corrects = 0.
            if phase == "train":
                model.train()
            else:
                model.eval()
            
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                
                with torch.autograd.set_grad_enabled(phase=="train"):
                    outputs = model(inputs) # [bsz, 2]
                    loss = loss_fn(outputs, labels)
                    
                preds = outputs.argmax(dim=1)
                if phase == "train":
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                running_loss += loss.item() * inputs.shape[0]
                running_corrects += torch.sum(preds.view(-1) == labels.view(-1)).item()
                
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects / len(dataloaders[phase].dataset)
            
            print("Phase {} loss:{}, acc:{}".format(phase, epoch_loss, epoch_acc))
            
            if phase == "val":
                val_acc_history.append(epoch_acc)
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [17]:
model_ft = model_ft.to(device)

optimizer = torch.optim.SGD(filter(lambda p:p.requires_grad, model_ft.parameters()),
                           lr=0.001,
                           momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [19]:
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
_, hist = train(model_ft, dataloader_dict, loss_fn, optimizer, num_epochs)

Phase train loss:0.6399044580146914, acc:0.5614754098360656
Phase val loss:0.4516520617054958, acc:0.8562091503267973
Phase train loss:0.5220197659046923, acc:0.7131147540983607
Phase val loss:0.34195033319635326, acc:0.9084967320261438
Phase train loss:0.37192933774385295, acc:0.8565573770491803
Phase val loss:0.27907954808933283, acc:0.9019607843137255
Phase train loss:0.3484264338602785, acc:0.8688524590163934
Phase val loss:0.244010944382038, acc:0.9411764705882353
Phase train loss:0.28509413902876807, acc:0.8975409836065574
Phase val loss:0.22908790282953798, acc:0.9281045751633987
Phase train loss:0.2598586150857269, acc:0.9139344262295082
Phase val loss:0.21317300726385677, acc:0.934640522875817
Phase train loss:0.24431768084158662, acc:0.9139344262295082
Phase val loss:0.20898831124399223, acc:0.9281045751633987
Phase train loss:0.20896800570800655, acc:0.9344262295081968
Phase val loss:0.19978281329659855, acc:0.9411764705882353
Phase train loss:0.21704333238914364, acc:0.9221